In [1]:
import pandas as pd 
import os
import json
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pickle

<img src="../../images/DER.jpg" width="600" height="350">

<div style="text-align:center;">
  <img src="../../images/DER.png" alt="DER" width="800" height="500">
</div>

La base de datos de yelp tiene la enterior estructura de relaciones. La idea es ir filtrando la información de forma que tengamos la información de los negocios que tenemos, es por esto que iniciaremos filtrando la base la tabla de businnes. luego analizaremos la tabla chechin, que tiene una relacion 1:1, para extrar solamente los checkin de negocios de interes, luego extraeremos los datos de reviews, donde tenemos una relación de 1:N, ya que un negoció puede tener muchas reviews, pero cada review hace referencia a un negicio particular, luego filtraremos la tabla users que se relaciona con **reviews** con una cardinalidad de 1:N y finalmente con la tabla tips. 

## Extracción 

En esta sección cargaremos el archivo business.pkl, el cual tiene un error de carga en que duplica cada columna, por lo que realizaremos el proceso de limpieza de este archivo, para posteriormente exportarlo en formate csv.gz

In [2]:
path_business = os.path.join('.','Datasets','Yelp','business.pkl')
business = pd.read_pickle(path_business)

business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,IN,T6J 5H2,53.468419,-113.492054,3.0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,DE,37204,36.115118,-86.766925,4.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,AB,46250,39.908707,-86.065088,3.5,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,AB,62025,38.782351,-89.950558,4.0,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Tranformación 

En esta sección eliminaremos las columnas duplicadas en el archivo cargado, ademas de filtrar los negocios de comidas rapidas **Fast food**

In [ ]:
business = business.iloc[:, :14]
business['categories'] = business['categories'].fillna('')
business = business.loc[business['categories'].str.contains('Fast food', case=False)]

In [5]:
business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,AZ,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,MO,37207,36.208102,-86.76817,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
97,knQ4vIgx-r85kjlWVVjcpQ,Silver and Blue Outfitters,5191 Meadowood Mall Cir,Reno,PA,89502,39.476518,-119.784037,4.5,5,1,"{'BikeParking': 'True', 'WiFi': 'u'no'', 'Dogs...","Sporting Goods, Sports Wear, Fashion, Restaura...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-21:0', ..."
122,VZzivJEbmHItOxNXJB3SpA,Subway,"330 Franklin Rd, Ste 900-d",Brentwood,TN,37027,36.026363,-86.789473,2.5,14,1,"{'BusinessParking': '{'garage': False, 'street...","Sandwiches, Fast Food, Restaurants","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ..."
153,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,10160 Ulmerton Rd,Largo,IN,33771,27.894167,-82.779866,3.0,19,1,"{'RestaurantsPriceRange2': '1', 'Caters': 'Tru...","Mexican, Fast Food, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '10:45-22:0',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150205,Agh0Z_PACmbdEpLyjvuwZg,Chick-fil-A,6240 E Whitestown Pkwy,Whitestown,FL,46075,39.949502,-86.349949,2.5,32,1,"{'BusinessParking': '{'garage': False, 'street...","Chicken Shop, Caterers, Fast Food, Restaurants...","{'Monday': '0:0-0:0', 'Tuesday': '6:30-21:0', ..."
150226,oNeZmmoO7OCyT2vSsVnGKw,Coney Island Sandwich Shop,250 Dr Martin Luther King Jr St N,St Petersburg,NJ,33705,27.77438,-82.646986,4.0,109,0,"{'RestaurantsPriceRange2': '1', 'RestaurantsDe...","Restaurants, Fast Food, Sandwiches, Hot Dogs","{'Monday': '10:0-16:0', 'Tuesday': '10:0-16:0'..."
150231,C96ynw3G1KrqBOdafxbTKw,Subway,4882 Sun City Center Blvd,Sun City Center,FL,33573,27.714605,-82.335851,4.0,5,1,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...","Restaurants, Fast Food, Sandwiches","{'Monday': '0:0-0:0', 'Tuesday': '9:0-21:0', '..."
150248,5ufcSii8MbAI3Yn5BQq2tA,Denny's,1096 Shighway Dr,Fenton,NJ,63026,38.541507,-90.467582,2.5,11,0,"{'GoodForKids': 'True', 'OutdoorSeating': 'Fal...","American (Traditional), Diners, Breakfast & Br...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


## Carga

Guardaremos el archivo en formato csv.gz

In [8]:
business_path = os.path.join('..','..','data','yelp','business.csv.gz')
business.to_csv(business_path,index=False,compression='gzip')